In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tsensor
from sklearn.preprocessing import StandardScaler

In [ ]:
# backward pass
def mse_grad(actual, predicted):
    return predicted - actual

# forward pass
inputs = np.array([[80], [90], [100], [-20], [-10]])
# instantiatind random weights and biases
# layer_1
l1_weights = np.array([[.82, .1]])
l1_bias = np.array([11.99, 10])
l1_output = inputs @ l1_weights + l1_bias
# applying relu activation
l1_activated = np.maximum(0, l1_output)
# layer_2
l2_weights = np.array([[0.5], [0.2]])
l2_bias = np.array([[5]])
output = l1_activated @ l2_weights + l2_bias
tmax = np.array([[80],[90], [100], [-20], [-10]])
tmax_tomorrow = np.array([[83], [89], [95], [-22], [-9]])

# back propagation
# partial derivative of loss wrt to the output layer2
output_gradient = mse_grad(tmax_tomorrow, output)
l2_w_gradient = l1_activated.T @ output_gradient
l2_b_gradient = np.mean(output_gradient, axis=0)
lr = 1e-4
l2_bias = l2_bias - l2_b_gradient * lr
l2_weigts = l2_weights - l2_w_gradient * lr

l1_activated_gradient = output_gradient @ l2_weights.T
l1_output_gradient = l1_activated_gradient * np.heaviside(l1_output, 0)

l1_w_gradient = inputs.T @ l1_output_gradient
l1_b_gradient = np.mean(l1_output_gradient, axis=0)

l1_weights -= l1_w_gradient * lr
l1_bias -= l1_b_gradient * lr

In [90]:
np.random.seed(42)
nn = [1, 3, 4, 2, 1]
inputs = np.random.randint(10, 100, 10).reshape(10, 1)
tagets = [n + np.random.randint(-5, 5) for n in inputs]
lr = 1e-5

# instantiation ransom weights and biases
np.random.seed(42)
layers = []
for i in range(1, len(nn)):
    layers.append([
        np.random.rand(nn[i-1], nn[i])/5 - .1,
        np.ones((1, nn[i]))
    ])
    
# forward pass
l1_output = np.maximum(0,inputs @ layers[0][0] + layers[0][1]) #hidden layer 1 output
l2_output = np.maximum(0, l1_output @ layers[1][0] + layers[1][1]) #hidden layer 2 output
l3_output = np.maximum(0, l2_output @ layers[2][0] + layers[2][1]) #hidden layer 3 output
output = l3_output @ layers[3][0] + layers[3][1] # output layers without relu

# back propagation and weight and bias updates
# from output to layer 4
output_gradient = mse_grad(tmax_tomorrow, output)
l4_w_gradient = l3_output.T @ output_gradient
l4_b_gradient = np.mean(output_gradient, axis=0)
layers[3][0] -= lr * l4_w_gradient
layers[3][1] -= lr * l4_b_gradient

# from layer4 to layer3
l3_gradient = output_gradient @ layers[3][0].T * np.heaviside(l3_output, 0)
l3_w_gradient = l2_output.T @ l3_gradient
l3_b_gradient = np.mean(l3_gradient, axis=0)
layers[2][0] -= lr * l3_w_gradient
layers[2][1] -= lr * l3_b_gradient

# form layer 3 to 2
l2_gradient = l3_gradient @ layers[2][0].T * np.heaviside(l2_output, 0)
l2_w_gradient = l1_output.T @ l2_gradient
l2_b_gradient = np.mean(l2_gradient, axis=0)
layers[1][0] -= lr * l2_w_gradient
layers[1][1] -= lr * l2_b_gradient

# from 2 to 1
l1_gradient = l2_gradient @ layers[1][0].T * np.heaviside(l1_output, 0)
l1_w_gradient = inputs.T @ l1_gradient
l1_b_gradient = np.mean(l1_gradient, axis=0)
layers[0][0] -= lr * l1_w_gradient
layers[0][1] -= lr * l1_b_gradient
